In [1]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving clouds.csv to clouds.csv
User uploaded file "clouds.csv" with length 872 bytes


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections
import warnings

from IPython.core.display import display, HTML
from sklearn.preprocessing import MinMaxScaler

# Import plotly
import plotly
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.offline as py
import plotly.tools as tls

# for color scales in plotly
import colorlover as cl


warnings.filterwarnings('ignore')

pd.options.display.max_columns = 999

py.init_notebook_mode(connected=True)

%load_ext autoreload
%autoreload 2
%matplotlib inline


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
df_orig = pd.read_csv('clouds.csv',
                     skiprows=0,
                     thousands=',',
                     )
df_orig

,Cloud Service,NVIDIA GPU,CPUs,GPU RAM,CPU RAM,Cost Per Hour,Wall Time,Cost to Train
0,Google Colab,K80,1,12.0,13,0.00,31.17,0.000
1,Google Cloud Compute Engine,P100,6,16.0,20,0.50,5.32,0.044
2,Google Cloud Compute Engine,K80,6,12.0,17,0.20,18.13,0.060
3,Google Cloud Compute Engine,V100,8,16.0,20,0.82,3.83,0.052
4,Google Cloud Compute Engine,P4,4,8.0,26,0.33,10.28,0.057
5,Google Cloud Compute Engine,V100 x 2,8,32.0,30,1.57,3.63,0.095
6,Google Cloud Compute Engine,V100 x 4,8,64.0,30,3.05,3.38,0.172
7,AWS EC2,K80 (p2.xlarge),4,12.0,61,0.28,20.90,0.098
8,AWS EC2,K80 x 8 (p2.8xlarge),32,96.0,488,2.35,16.12,0.631
9,AWS EC2,V100 (p3.2xlarge),8,16.0,61,1.05,3.85,0.067


In [0]:
df = df_orig.copy()

In [6]:
df

,Cloud Service,NVIDIA GPU,CPUs,GPU RAM,CPU RAM,Cost Per Hour,Wall Time,Cost to Train
0,Google Colab,K80,1,12.0,13,0.00,31.17,0.000
1,Google Cloud Compute Engine,P100,6,16.0,20,0.50,5.32,0.044
2,Google Cloud Compute Engine,K80,6,12.0,17,0.20,18.13,0.060
3,Google Cloud Compute Engine,V100,8,16.0,20,0.82,3.83,0.052
4,Google Cloud Compute Engine,P4,4,8.0,26,0.33,10.28,0.057
5,Google Cloud Compute Engine,V100 x 2,8,32.0,30,1.57,3.63,0.095
6,Google Cloud Compute Engine,V100 x 4,8,64.0,30,3.05,3.38,0.172
7,AWS EC2,K80 (p2.xlarge),4,12.0,61,0.28,20.90,0.098
8,AWS EC2,K80 x 8 (p2.8xlarge),32,96.0,488,2.35,16.12,0.631
9,AWS EC2,V100 (p3.2xlarge),8,16.0,61,1.05,3.85,0.067


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 8 columns):
Cloud Service    15 non-null object
NVIDIA GPU       15 non-null object
CPUs             15 non-null int64
GPU RAM          15 non-null float64
CPU RAM          15 non-null int64
Cost Per Hour    15 non-null float64
Wall Time        14 non-null float64
Cost to Train    15 non-null float64
dtypes: float64(4), int64(2), object(2)
memory usage: 1.0+ KB


In [8]:
df['full'] = df['Cloud Service'] + "" + df['NVIDIA GPU']
df.head(3)

,Cloud Service,NVIDIA GPU,CPUs,GPU RAM,CPU RAM,Cost Per Hour,Wall Time,Cost to Train,full
0,Google Colab,K80,1,12.0,13,0.0,31.17,0.000,Google ColabK80
1,Google Cloud Compute Engine,P100,6,16.0,20,0.5,5.32,0.044,Google Cloud Compute EngineP100
2,Google Cloud Compute Engine,K80,6,12.0,17,0.2,18.13,0.060,Google Cloud Compute EngineK80


In [9]:
df.set_index('full', inplace=True)
df

,Cloud Service,NVIDIA GPU,CPUs,GPU RAM,CPU RAM,Cost Per Hour,Wall Time,Cost to Train
full,,,,,,,,
Google ColabK80,Google Colab,K80,1,12.0,13,0.00,31.17,0.000
Google Cloud Compute EngineP100,Google Cloud Compute Engine,P100,6,16.0,20,0.50,5.32,0.044
Google Cloud Compute EngineK80,Google Cloud Compute Engine,K80,6,12.0,17,0.20,18.13,0.060
Google Cloud Compute EngineV100,Google Cloud Compute Engine,V100,8,16.0,20,0.82,3.83,0.052
Google Cloud Compute EngineP4,Google Cloud Compute Engine,P4,4,8.0,26,0.33,10.28,0.057
Google Cloud Compute EngineV100 x 2,Google Cloud Compute Engine,V100 x 2,8,32.0,30,1.57,3.63,0.095
Google Cloud Compute EngineV100 x 4,Google Cloud Compute Engine,V100 x 4,8,64.0,30,3.05,3.38,0.172
AWS EC2K80 (p2.xlarge),AWS EC2,K80 (p2.xlarge),4,12.0,61,0.28,20.90,0.098
AWS EC2K80 x 8 (p2.8xlarge),AWS EC2,K80 x 8 (p2.8xlarge),32,96.0,488,2.35,16.12,0.631


In [12]:
df = df.sort_values(by = ['Cost to Train'])
df.head(5)

,Cloud Service,NVIDIA GPU,CPUs,GPU RAM,CPU RAM,Cost Per Hour,Wall Time,Cost to Train
full,,,,,,,,
Google ColabK80,Google Colab,K80,1,12.0,13,0.00,31.17,0.000
KaggleK80,Kaggle,K80,2,12.0,6,0.00,NaN,0.000
vast.aiGTX 1070 Ti,vast.ai,GTX 1070 Ti,4,8.1,16,0.06,7.23,0.008
Google Cloud Compute EngineP100,Google Cloud Compute Engine,P100,6,16.0,20,0.50,5.32,0.044
Google Cloud Compute EngineV100,Google Cloud Compute Engine,V100,8,16.0,20,0.82,3.83,0.052


Let's make our first chart in Plotly. Let's chart the cost to train.

In [14]:
cmax=200
cmin=50
color_s = np.linspace(cmin, cmax, 13)

data = [
    go.Bar(
        x=df.index,          
        y=df['Cost to Train'],
        marker=dict(
            colorscale='Jet',
            color=color_s,
        ),
    )
]

layout = {
    'title': 'Cost to Train',
    'xaxis': {'title': 'Cloud Software', 'tickmode': 'linear'},
    'yaxis': {'title': "$ USD"}
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [15]:
df = df.iloc[:-2]
df

,Cloud Service,NVIDIA GPU,CPUs,GPU RAM,CPU RAM,Cost Per Hour,Wall Time,Cost to Train
full,,,,,,,,
Google ColabK80,Google Colab,K80,1,12.0,13,0.00,31.17,0.000
KaggleK80,Kaggle,K80,2,12.0,6,0.00,NaN,0.000
vast.aiGTX 1070 Ti,vast.ai,GTX 1070 Ti,4,8.1,16,0.06,7.23,0.008
Google Cloud Compute EngineP100,Google Cloud Compute Engine,P100,6,16.0,20,0.50,5.32,0.044
Google Cloud Compute EngineV100,Google Cloud Compute Engine,V100,8,16.0,20,0.82,3.83,0.052
Google Cloud Compute EngineP4,Google Cloud Compute Engine,P4,4,8.0,26,0.33,10.28,0.057
Google Cloud Compute EngineK80,Google Cloud Compute Engine,K80,6,12.0,17,0.20,18.13,0.060
AWS EC2V100 (p3.2xlarge),AWS EC2,V100 (p3.2xlarge),8,16.0,61,1.05,3.85,0.067
PaperspaceQuadro M4000,Paperspace,Quadro M4000,8,8.0,30,0.51,8.30,0.071


In [18]:
labs = [
    'Google Colab K80',
    'vast.ai GTX1070Ti',
    'GCP P100',
    'GCP V100',
    'GCP P4',
    'GCP K80',
    'AWS EC2 V100',
    'Paperspace M4000',
    'GCP V100 x2',
    'AWS EC2 K80',
    'GCP V100 x4',
    'AWS EC2 V100 x4',
    'Kaggle',
]

len(labs)

13

In [19]:
df['short_names'] = labs
df

,Cloud Service,NVIDIA GPU,CPUs,GPU RAM,CPU RAM,Cost Per Hour,Wall Time,Cost to Train,short_names
full,,,,,,,,,
Google ColabK80,Google Colab,K80,1,12.0,13,0.00,31.17,0.000,Google Colab K80
KaggleK80,Kaggle,K80,2,12.0,6,0.00,NaN,0.000,vast.ai GTX1070Ti
vast.aiGTX 1070 Ti,vast.ai,GTX 1070 Ti,4,8.1,16,0.06,7.23,0.008,GCP P100
Google Cloud Compute EngineP100,Google Cloud Compute Engine,P100,6,16.0,20,0.50,5.32,0.044,GCP V100
Google Cloud Compute EngineV100,Google Cloud Compute Engine,V100,8,16.0,20,0.82,3.83,0.052,GCP P4
Google Cloud Compute EngineP4,Google Cloud Compute Engine,P4,4,8.0,26,0.33,10.28,0.057,GCP K80
Google Cloud Compute EngineK80,Google Cloud Compute Engine,K80,6,12.0,17,0.20,18.13,0.060,AWS EC2 V100
AWS EC2V100 (p3.2xlarge),AWS EC2,V100 (p3.2xlarge),8,16.0,61,1.05,3.85,0.067,Paperspace M4000
PaperspaceQuadro M4000,Paperspace,Quadro M4000,8,8.0,30,0.51,8.30,0.071,GCP V100 x2


In [20]:
import matplotlib.pyplot as plt
%matplotlib inline

cmax=200
cmin=50
color_s = np.linspace(cmin, cmax, 12)

data = [
    go.Bar(
        x=df.short_names,          
        y=df['Cost to Train'],
        marker=dict(
            colorscale='Jet',
            # cauto=True,
            color=color_s,
        ),
    )
]

layout = {
    'title': 'Cost to Train',
    'xaxis': {'title': 'Cloud Software', 'tickmode': 'linear'},
    'yaxis': {'title': "$ USD"}
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)


In [21]:
df = df.sort_values(by = ['Wall Time'])
df

,Cloud Service,NVIDIA GPU,CPUs,GPU RAM,CPU RAM,Cost Per Hour,Wall Time,Cost to Train,short_names
full,,,,,,,,,
AWS EC2V100 x 4 (p3.8xlarge),AWS EC2,V100 x 4 (p3.8xlarge),64,128.0,488,4.05,2.97,0.200,Kaggle
Google Cloud Compute EngineV100 x 4,Google Cloud Compute Engine,V100 x 4,8,64.0,30,3.05,3.38,0.172,AWS EC2 V100 x4
Google Cloud Compute EngineV100 x 2,Google Cloud Compute Engine,V100 x 2,8,32.0,30,1.57,3.63,0.095,AWS EC2 K80
Google Cloud Compute EngineV100,Google Cloud Compute Engine,V100,8,16.0,20,0.82,3.83,0.052,GCP P4
AWS EC2V100 (p3.2xlarge),AWS EC2,V100 (p3.2xlarge),8,16.0,61,1.05,3.85,0.067,Paperspace M4000
Google Cloud Compute EngineP100,Google Cloud Compute Engine,P100,6,16.0,20,0.50,5.32,0.044,GCP V100
vast.aiGTX 1070 Ti,vast.ai,GTX 1070 Ti,4,8.1,16,0.06,7.23,0.008,GCP P100
PaperspaceQuadro M4000,Paperspace,Quadro M4000,8,8.0,30,0.51,8.30,0.071,GCP V100 x2
Google Cloud Compute EngineP4,Google Cloud Compute Engine,P4,4,8.0,26,0.33,10.28,0.057,GCP K80


In [22]:
cmax=200
cmin=50
color_s = np.linspace(cmin, cmax, 12)

data = [
    go.Bar(
        x=df.short_names,          
        y=df['Wall Time'],
        marker=dict(
            colorscale='Jet',
            color=color_s,
        ),
    )
]

layout = {
    'title': 'Time to Train',
    'xaxis': {'title': 'Cloud Software', 'tickmode': 'linear'},
    'yaxis': {'title': "Minutes"}
}

fig = go.Figure(data=data, layout=layout)
py.iplot(fig)

In [23]:
df_best = df.iloc[[0,1,2,4,5,6,11],:]
df_best

,Cloud Service,NVIDIA GPU,CPUs,GPU RAM,CPU RAM,Cost Per Hour,Wall Time,Cost to Train,short_names
full,,,,,,,,,
AWS EC2V100 x 4 (p3.8xlarge),AWS EC2,V100 x 4 (p3.8xlarge),64,128.0,488,4.05,2.97,0.200,Kaggle
Google Cloud Compute EngineV100 x 4,Google Cloud Compute Engine,V100 x 4,8,64.0,30,3.05,3.38,0.172,AWS EC2 V100 x4
Google Cloud Compute EngineV100 x 2,Google Cloud Compute Engine,V100 x 2,8,32.0,30,1.57,3.63,0.095,AWS EC2 K80
AWS EC2V100 (p3.2xlarge),AWS EC2,V100 (p3.2xlarge),8,16.0,61,1.05,3.85,0.067,Paperspace M4000
Google Cloud Compute EngineP100,Google Cloud Compute Engine,P100,6,16.0,20,0.50,5.32,0.044,GCP V100
vast.aiGTX 1070 Ti,vast.ai,GTX 1070 Ti,4,8.1,16,0.06,7.23,0.008,GCP P100
Google ColabK80,Google Colab,K80,1,12.0,13,0.00,31.17,0.000,Google Colab K80


In [24]:
df_aws = df[df['Cloud Service']=='AWS EC2']
df_aws

,Cloud Service,NVIDIA GPU,CPUs,GPU RAM,CPU RAM,Cost Per Hour,Wall Time,Cost to Train,short_names
full,,,,,,,,,
AWS EC2V100 x 4 (p3.8xlarge),AWS EC2,V100 x 4 (p3.8xlarge),64,128.0,488,4.05,2.97,0.200,Kaggle
AWS EC2V100 (p3.2xlarge),AWS EC2,V100 (p3.2xlarge),8,16.0,61,1.05,3.85,0.067,Paperspace M4000
AWS EC2K80 (p2.xlarge),AWS EC2,K80 (p2.xlarge),4,12.0,61,0.28,20.90,0.098,GCP V100 x4


In [25]:
df_gcp = df[df['Cloud Service']=='Google Cloud Compute Engine']
df_gcp

,Cloud Service,NVIDIA GPU,CPUs,GPU RAM,CPU RAM,Cost Per Hour,Wall Time,Cost to Train,short_names
full,,,,,,,,,
Google Cloud Compute EngineV100 x 4,Google Cloud Compute Engine,V100 x 4,8,64.0,30,3.05,3.38,0.172,AWS EC2 V100 x4
Google Cloud Compute EngineV100 x 2,Google Cloud Compute Engine,V100 x 2,8,32.0,30,1.57,3.63,0.095,AWS EC2 K80
Google Cloud Compute EngineV100,Google Cloud Compute Engine,V100,8,16.0,20,0.82,3.83,0.052,GCP P4
Google Cloud Compute EngineP100,Google Cloud Compute Engine,P100,6,16.0,20,0.50,5.32,0.044,GCP V100
Google Cloud Compute EngineP4,Google Cloud Compute Engine,P4,4,8.0,26,0.33,10.28,0.057,GCP K80
Google Cloud Compute EngineK80,Google Cloud Compute Engine,K80,6,12.0,17,0.20,18.13,0.060,AWS EC2 V100


In [26]:
df_vast = df[df['Cloud Service'] =='vast.ai']
df_vast

,Cloud Service,NVIDIA GPU,CPUs,GPU RAM,CPU RAM,Cost Per Hour,Wall Time,Cost to Train,short_names
full,,,,,,,,,
vast.aiGTX 1070 Ti,vast.ai,GTX 1070 Ti,4,8.1,16,0.06,7.23,0.008,GCP P100


In [27]:
dot_size = 10

trace1 = go.Scatter(
    x = df_aws['Wall Time'],
    y = df_aws['Cost Per Hour'],
    marker = {'color': 'purple', 'size':dot_size},
    mode = 'markers+text',
    text = df_aws['short_names'],
    name = 'AWS',
    textposition = 'top center'
 )

# the line
trace2 = go.Scatter(
    x=df_best['Wall Time'], 
    y=df_best['Cost Per Hour'],
    marker={'color': 'yellow',}, 
    mode="lines",  
    name='Efficient Frontier',
)

trace3 = go.Scatter(
    x=df_colab['Wall Time'], 
    y=df_colab['Cost Per Hour'],
    marker={'color': 'red', 'size': dot_size}, 
    mode="markers+text",  
    text=df_colab['short_names'],
    name="Google Colab",
    textposition="top center"
)

trace4 = go.Scatter(
    x=df_paper['Wall Time'], 
    y=df_paper['Cost Per Hour'],
    marker={'color': 'green', 'size': dot_size}, 
    mode="markers+text",  
    text=df_paper['short_names'],
    name="Paperspace",
    textposition="top center"
)

trace5 = go.Scatter(
    x=df_vast['Wall Time'], 
    y=df_vast['Cost Per Hour'],
    marker={'color': 'blue', 'size': dot_size}, 
    mode="markers+text",  
    text=df_vast['short_names'],
    name="vast.ai",
    textposition="top center"
)

trace6 = go.Scatter(
    x=df_gcp['Wall Time'], 
    y=df_gcp['Cost Per Hour'],
    marker={'color': 'black', 'size': dot_size}, 
    mode="markers+text",  
    text=df_gcp.short_names,
    name="Google Cloud",
    textposition="top center"
)

data = [trace1, trace2, trace3, trace4, trace5, trace6]

layout = go.Layout(title='Cost per Hour vs Time to Train')

fig = go.Figure(data, layout)

py.iplot(fig)
fig

NameError: ignored